In [56]:
import pickle
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.covariance import EmpiricalCovariance
import torch
#from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE

In [ ]:
with open('classwise_GTfeatures.pkl', 'rb') as f:
        embed_classwise = pickle.load(f)

In [2]:
embed_classwise = {}
fea_path = "/home/vaibhav/Desktop/stud/datasets/esmart/bigdetect_features/GT/pooled/"

In [3]:
for im in os.listdir(fea_path):
    x = {}
    with open(os.path.join(fea_path,im), 'rb') as f:
        x = pickle.load(f)
        
    idx = 0
    for i in x['preds']:
        if embed_classwise.get(i.item()) == None:
            embed_classwise[i.item()] = [x['features'][idx].detach().cpu().numpy()]
        else:
            embed_classwise[i.item()].append(x['features'][idx].detach().cpu().numpy())
        idx += 1

In [13]:
a = np.array(embed_classwise[5])

In [14]:
a.shape

(165, 1024)

In [19]:
embed_classwise[0][0].shape

(1024,)

In [49]:
classwise_mean = np.zeros((len(embed_classwise.keys()),embed_classwise[0][0].shape[0]),dtype='float32')
precision = []
for c in range(len(embed_classwise.keys())):
    f_x = np.array(embed_classwise[c])
    classwise_mean[c] = f_x.mean(axis=0)
    f_x = f_x - classwise_mean[c][np.newaxis,:]
    print(f_x.shape)
    precision.append(f_x)

(170, 1024)
(392, 1024)
(42233, 1024)
(625, 1024)
(7420, 1024)
(165, 1024)
(1573, 1024)
(7762, 1024)
(6797, 1024)
(1032, 1024)
(4381, 1024)
(4470, 1024)


In [50]:
precision = np.concatenate(precision,axis=0)

In [51]:
precision.shape

(77020, 1024)

In [28]:
classwise_mean[0]

array([0.00576049, 0.12466978, 0.45142594, ..., 0.0345595 , 0.42663744,
       1.5739923 ], dtype=float32)

In [45]:
a = np.array(embed_classwise[0]) - classwise_mean[0][np.newaxis,:]

In [47]:
a.shape

(170, 1024)

In [58]:
group_lasso = EmpiricalCovariance(assume_centered=False)
final_precision = []
group_lasso.fit(precision)
temp_precision = group_lasso.precision_
#temp_precision = torch.from_numpy(temp_precision).float().cuda()
final_precision.append(temp_precision)

In [60]:
final_precision[0].shape

(1024, 1024)

In [62]:
temp_precision.shape

(1024, 1024)

In [65]:
def sample_estimator(embed_classwise):
    classwise_mean = np.zeros((len(embed_classwise.keys()),embed_classwise[0][0].shape[0]),dtype='float32')
    precision = []
    group_lasso = EmpiricalCovariance(assume_centered=False)
    for c in range(len(embed_classwise.keys())):
        f_x = np.array(embed_classwise[c])
        classwise_mean[c] = f_x.mean(axis=0)
        f_x = f_x - classwise_mean[c][np.newaxis,:]
        
        ### finding inverse of covariance
        group_lasso.fit(f_x)
        temp_precision = group_lasso.precision_
        precision.append(temp_precision)
    precision = np.array(precision)
   

    return classwise_mean,precision

In [66]:
sample_mean,cov = sample_estimator(embed_classwise)

In [67]:
sample_mean.shape

(12, 1024)

In [68]:
cov.shape

(12, 1024, 1024)